# Credit Risk Resampling Techniques

In [2]:
# For output clarity 
import warnings
warnings.filterwarnings('ignore')

In [23]:
# For cleaning data
import numpy as np
import pandas as pd

# For creating training and testing sets
from sklearn.model_selection import train_test_split

# For confirming im/balance
from collections import Counter

# For creating modles
from sklearn.linear_model import LogisticRegression

# For validating models
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Reading the CSV and Performing Data Preprocessing 

In [37]:
#Set up import of columns
#Columns were at first chosen through visual evaluation. 
#If a column contained all '0' values or all '1' values, it was not included
#If a column contained the same value for every row, it was not included 
columns = [
    "loan_amnt", "term", "int_rate", "installment", "grade",
    "home_ownership", "annual_inc", "verification_status", "issue_d","loan_status",
    "dti", "delinq_2yrs", "inq_last_6mths", "open_acc", "pub_rec",
    "revol_bal", "total_acc", "initial_list_status", "out_prncp", "out_prncp_inv", 
    "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int","last_pymnt_amnt", 
    "next_pymnt_d", "collections_12_mths_ex_med", "application_type", "tot_coll_amt", "tot_cur_bal",
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", "mths_since_rcnt_il",
    "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m", 
    "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", 
    "delinq_amnt", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", 
    "mort_acc", "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl", 
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl", 
    "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", "num_tl_90g_dpd_24m", "num_tl_op_past_12m", 
    "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies", "tot_hi_cred_lim", "total_bal_ex_mort", 
    "total_bc_limit", "total_il_high_credit_limit"
]

target = ["loan_status"]

In [38]:
# Load the data
file_path = 'LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Any coloumns where all values were null we dropped
df = df.dropna(axis='columns', how='all')

# Any rows that contained null values were dropped
df = df.dropna()

# The `Issued` loan status was removed from the loan_status column
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# The interest rate was converted to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# The target column values were converted to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df

,loan_amnt,term,int_rate,installment,grade,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,application_type,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,36 months,0.1719,375.35,C,RENT,66000.0,Source Verified,Mar-2019,low_risk,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,w,10204.87,10204.87,355.29,355.29,295.13,60.16,375.35,May-2019,0.0,Individual,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,59.0,1.0,2.0,1553.0,58.0,3700.0,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,36 months,0.2000,929.09,D,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,w,24293.13,24293.13,873.53,873.53,706.87,166.66,929.09,May-2019,0.0,Individual,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,86.0,4.0,8.0,5197.0,61.0,50400.0,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,60 months,0.2000,529.88,D,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,w,19636.79,19636.79,485.44,485.44,363.21,122.23,529.88,May-2019,0.0,Individual,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,75.0,2.0,3.0,2810.0,75.0,17500.0,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,36 months,0.1640,353.55,C,RENT,92000.0,Verified,Mar-2019,low_risk,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,w,9719.34,9719.34,335.33,335.33,280.66,54.67,353.55,May-2019,0.0,Individual,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,66.0,0.0,1.0,6360.0,69.0,23000.0,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,60 months,0.1474,520.39,C,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,w,21614.73,21614.73,484.36,484.36,385.27,99.09,520.39,May-2019,0.0,Individual,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,84.0,1.0,1.0,3600.0,41.0,43100.0,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71022,12000.0,60 months,0.2727,368.37,E,RENT,63000.0,Not Verified,Jan-2019,low_risk,29.07,0.0,0.0,8.0,0.0,13314.0,26.0,w,11706.42,11706.42,1086.93,1086.93,293.58,793.35,368.37,May-2019,0.0,Individual,0.0,60350.0,3.0,4.0,2.0,2.0,2.0,47036.0,75.0,3.0,5.0,1177.0,69.0,25000.0,1.0,0.0,4.0,7.0,7544.0,11387.0,15.7,0.0,0.0,164.0,107.0,1.0,1.0,0.0,1.0,2.0,0.0,3.0,4.0,3.0,4.0,18.0,4.0,8.0,4.0,8.0,

In [39]:
#df.columns

In [40]:
#Encode feature columns with string values 
df_binary_encoded = pd.get_dummies(df, columns = [
    "term",
    "grade",
    "home_ownership",
    "issue_d",
    "verification_status",
    "initial_list_status",
    "next_pymnt_d",
    "application_type"
])
#Encode target column
#low_risk: 0
#high_risk: 1
x = {'low_risk': 0}   
df_binary_encoded = df_binary_encoded.replace(x)
y = {'high_risk': 1}
df_binary_encoded = df_binary_encoded.replace(y)

pd.set_option('display.max_columns', None)
df_binary_encoded

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_amnt,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,0,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,10204.87,10204.87,355.29,355.29,295.13,60.16,375.35,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,59.0,1.0,2.0,1553.0,58.0,3700.0,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,0,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,24293.13,24293.13,873.53,873.53,706.87,166.66,929.09,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,86.0,4.0,8.0,5197.0,61.0,50400.0,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,0,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,19636.79,19636.79,485.44,485.44,363.21,122.23,529.88,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,75.0,2.0,3.0,2810.0,75.0,17500.0,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,0,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,9719.34,9719.34,335.33,335.33,280.66,54.67,353.55,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,66.0,0.0,1.0,6360.0,69.0,23000.0,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,0,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,21614.73,21614.73,484.36,484.36,385.27,99.09,520.39,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,84.0,1.0,1.0,3600.0,41.0,43100.0,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71022,12000.0,0.2727,368.37,63000.0,0,29.07,0.0,0.0,8.0,0

# Split the Data into Training and Testing

In [41]:
# Creating the features
X = df_binary_encoded.drop(columns="loan_status")

# Creating the target
y = df_binary_encoded["loan_status"]

In [42]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_amnt,collections_12_mths_ex_med,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
count,71027.000000,71027.000000,71027.000000,7.102700e+04,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,7.102700e+04,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,7.102700e+04,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,7.102700e+04,7.102700e+04,71027.000000,7.102700e+04,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000,71027.000000
mean,16701.921452,0.128636,482.253956,8.852203e+04,21.756772,0.225647,0.510200,12.571248,0.125121,18017.618666,24.587678,16045.674304,16042.767830,970.145755,969.939534,656.247169,313.863546,518.210138,0.017303,185.759514,1.640824e+05,1.017247,3.274572,0.837245,1.889690,14.490166,4.303074e+04,69.083236,1.236262,2.621440,6384.817647,55.941557,42599.212989,1.346418,1.660003,2.175637,4.804947,14203.863643,16650.838076,49.016423,0.007223,3.068537,125.561111,175.674222,14.253467,7.195686,1.439917,24.560407,7.646993,0.463232,3.850888,5.736466,5.197432,7.421558,9.431216,8.626438,13.514635,5.684909,12.552255,0.055796,2.207879,94.987733,31.486439,0.125065,2.104007e+05,6.137732e+04,29801.752094,5.546523e+04,0.671027,0.328973,0.305264,0.289355,0.241077,0.129064,0.034888,0.000225,0.000127,0.009025,0.526884,0.106790,0.357301,0.372577,0.448548,0.178876,0.474566,0.370225,0.155209,0.123657,0.876343,0.383826,0.616174,0.864685,0.135315
std,10253.207248,0.048335,288.584068,1.146235e+05,19.936996,0.737205,0.772191,6.013969,0.335927,22640.570431,12.273987,9979.726786,9981.967883,1230.044974,1230.050047,1126.092139,281.300162,808.944703,0.142375,1785.519606,1.731191e+05,1.173295,2.959005,0.966599,1.537591,14.532706,4.943571e+04,22.637551,1.466107,2.494872,5791.322093,18.721044,37585.839908,1.535267,2.830772,2.370713,3.132291,15934.821351,20230.827789

In [43]:
# Checking the balance of the target values
#low_risk: 0
#high_risk: 1
y.value_counts()

0    70669
1      358
Name: loan_status, dtype: int64

In [44]:
# Normal train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#Confirming imbalance in the training set
Counter(y_train)

Counter({0: 52996, 1: 274})

# Oversampling

Oversampling the data using the naive random oversampling algorithm and the SMOTE algorithm. 

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [45]:
# Resampling the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#Confirming enlargement of the minority class
Counter(y_resampled)

Counter({0: 52996, 1: 52996})

In [46]:
# Training the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [48]:
# Generating preciditions with trained model
y_pred = model.predict(X_test)

In [49]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.663038250438522

In [50]:
# Displaying confusion matrix
confusion_matrix(y_test, y_pred)

array([[10181,  7492],
       [   21,    63]])

In [52]:
# Displaying the imbalanced classification report
#low_risk: 0
#high_risk: 1
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.58      0.75      0.73      0.66      0.42     17673
          1       0.01      0.75      0.58      0.02      0.66      0.44        84

avg / total       0.99      0.58      0.75      0.73      0.66      0.42     17757



### SMOTE Oversampling

In [54]:
# Resampling the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
#Confirming enlargement of the minority class
Counter(y_resampled)

Counter({0: 52996, 1: 52996})

In [55]:
# Training the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [56]:
# Generating preciditions with trained model
y_pred = model.predict(X_test)

In [57]:
# Calculating the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6761322760304258

In [58]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_pred)

array([[11275,  6398],
       [   24,    60]])

In [59]:
# Displaying the imbalanced classification report
#low_risk: 0
#high_risk: 1
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.64      0.71      0.78      0.68      0.45     17673
          1       0.01      0.71      0.64      0.02      0.68      0.46        84

avg / total       0.99      0.64      0.71      0.77      0.68      0.45     17757



# Undersampling

Undersampling the data using the Cluster Centroids algorithm.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [61]:
# Resampling the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# Verifying reduction of the majority class
Counter(y_resampled)

Counter({0: 274, 1: 274})

In [62]:
# Training the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [63]:
# Generating preciditions with trained model
y_pred = model.predict(X_test)

In [64]:
# Calculating the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.4903434213610754

In [65]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_pred)

array([[7864, 9809],
       [  39,   45]])

In [66]:
# Displaying the imbalanced classification report
#low_risk: 0
#high_risk: 1
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.44      0.54      0.61      0.49      0.24     17673
          1       0.00      0.54      0.44      0.01      0.49      0.24        84

avg / total       0.99      0.45      0.54      0.61      0.49      0.24     17757



# Combination (Over and Under) Sampling
Resampling the data using the SMOTEENN algorithm

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [67]:
# Resampling the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

#Confirming balance
Counter(y_resampled)

Counter({0: 64034, 1: 70654})

In [68]:
# Training the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [69]:
# Generating preciditions with trained model
y_pred = model.predict(X_test)

In [70]:
# Calculating the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.680895393295665

In [71]:
# Displaying the confusion matrix
confusion_matrix(y_test, y_pred)

array([[10181,  7492],
       [   18,    66]])

In [72]:
# Displaying the imbalanced classification report
#low_risk: 0
#high_risk: 1
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.58      0.79      0.73      0.67      0.44     17673
          1       0.01      0.79      0.58      0.02      0.67      0.46        84

avg / total       0.99      0.58      0.78      0.73      0.67      0.44     17757

